In [55]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import json
import cv2
import numpy as np

In [56]:
devic = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [63]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

In [64]:
train_dataset = KeypointsDataset("data/images","data/data_train.json")
val_dataset = KeypointsDataset("data/images","data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

In [65]:
#Creat model
model = models.resnet50(pretrained=True)
model.fc =  torch.nn.Linear(model.fc.in_features, 14*2) #Replaces last layer

In [66]:
model = model.to(device)

In [67]:
#Train model
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [68]:
epochs=20
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

Epoch 0, iter 0, loss: 14763.46875
Epoch 0, iter 10, loss: 14929.2509765625
Epoch 0, iter 20, loss: 13480.7373046875
Epoch 0, iter 30, loss: 13763.3603515625
Epoch 0, iter 40, loss: 13741.3974609375
Epoch 0, iter 50, loss: 12806.685546875
Epoch 0, iter 60, loss: 12629.330078125
Epoch 0, iter 70, loss: 12672.060546875
Epoch 0, iter 80, loss: 11593.9697265625
Epoch 0, iter 90, loss: 11828.8984375
Epoch 0, iter 100, loss: 11329.080078125
Epoch 0, iter 110, loss: 10820.6474609375
Epoch 0, iter 120, loss: 10793.7431640625
Epoch 0, iter 130, loss: 9414.7255859375
Epoch 0, iter 140, loss: 9126.677734375
Epoch 0, iter 150, loss: 8881.806640625
Epoch 0, iter 160, loss: 8728.2021484375
Epoch 0, iter 170, loss: 8850.1826171875
Epoch 0, iter 180, loss: 8818.1240234375
Epoch 0, iter 190, loss: 8390.8818359375
Epoch 0, iter 200, loss: 7882.5859375
Epoch 0, iter 210, loss: 7614.83251953125
Epoch 0, iter 220, loss: 7091.40625
Epoch 0, iter 230, loss: 7237.45166015625
Epoch 0, iter 240, loss: 6500.0102

In [69]:
torch.save(model.state_dict(), "keypoints_model.pth")